In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
import json 
import re
import random
random.seed(28)
import keras
from keras.layers import *
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

In [ ]:
def load_data():
    legit_file = open('/Users/s0c02nj/Downloads/legitimate_url.json', 'r') 
    legit_data = json.load(legit_file)

    phishing_file = open('/Users/s0c02nj/Downloads/phishing_url.json', 'r') 
    phishing_data = json.load(phishing_file)

    df_1 = pd.DataFrame(legit_data, columns = ['url'])
    df_1['is_phishing'] = 0

    df_2 = pd.DataFrame(phishing_data, columns = ['url'])
    df_2['is_phishing'] = 1

    df = pd.concat([df_1,df_2], axis = 0)
    del df_1, df_2
    
    return df


In [ ]:
df_url = load_data()

In [ ]:
df_url['is_phishing'].value_counts()

In [ ]:
#df['url'].iloc[27777]

In [ ]:
urls = list(df_url['url'].values)

In [ ]:
char2idx = dict()
max_url_seq_length = 0

for url in tqdm(urls):
    max_url_seq_length = max(max_url_seq_length, len(url))
    for c in url:
        if c not in char2idx:
            char2idx[c] = len(char2idx)
num_input_tokens = len(char2idx)
idx2char = dict([(idx, c) for c, idx in char2idx.items()])

config = dict()
config['num_input_tokens'] = num_input_tokens
config['char2idx'] = char2idx
config['idx2char'] = idx2char
config['max_url_seq_length'] = max_url_seq_length

In [ ]:
# config['max_url_seq_length']

In [ ]:
data_size = df_url.shape[0]
X = np.zeros(shape=(data_size, max_url_seq_length))
Y = keras.utils.to_categorical(df_url['is_phishing'])


In [ ]:
for i in tqdm(range(data_size)):
    url = df_url['url'].iloc[i]
    #label = df_url['is_phishing'].iloc[i]
    for idx, c in enumerate(url):
        X[i, idx] = char2idx[c]
    

In [ ]:
X

In [ ]:
num_input_tokens = config['num_input_tokens']
char2idx = config['char2idx']
idx2char = config['idx2char']
max_url_seq_length = config['max_url_seq_length']

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, 
                                                test_size=0.2, 
                                                stratify = Y,
                                                random_state=42)

#### Bi-LSTM Model

In [ ]:
EMBEDDING_SIZE = 100

In [ ]:
def make_bidirectional_lstm_model(num_input_tokens, max_len):
    model = Sequential()
    model.add(Embedding(input_dim = num_input_tokens, 
                        output_dim= EMBEDDING_SIZE, 
                        input_length=max_len))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(units=64, dropout=0.2, 
                                 recurrent_dropout=0.2, 
                                 input_shape=(max_len, EMBEDDING_SIZE))))
    model.add(Dense(2, activation='softmax'))
    return model


In [ ]:
num_input_tokens = num_input_tokens
max_len = max_url_seq_length

In [ ]:
model = make_bidirectional_lstm_model(num_input_tokens, max_len)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 512
epochs = 5

In [ ]:
history = model.fit(Xtrain, Ytrain, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1,
                    validation_data=(Xtest, Ytest), 
                   )

In [2]:
import tensorflow as tf
import keras

In [3]:
tf.__version__

'1.13.1'

In [4]:
keras.__version__

'2.2.4'